In [1]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:5 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,951 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,561 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,076 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restr

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
# Initialize findspark
import findspark
findspark.init()

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SQL operations in pyspark").getOrCreate()

data = "/content/drive/MyDrive/Colab Notebooks/bfsi_models_practice/pyspark/sales_data.csv"
df = spark.read.options(header=True, inferSchema=True).csv(data)
df.createOrReplaceTempView("sales_data")

In [12]:
result = spark.sql('select * from sales_data')
result.show()

+-------+---------+--------+-----+----------+
|OrderID|ProductID|Quantity|Price| OrderDate|
+-------+---------+--------+-----+----------+
|      1|      101|       3|  100|01-01-2023|
|      2|      102|       1|  200|02-01-2023|
|      3|      101|       2|  100|03-01-2023|
|      4|      103|       5|   50|04-01-2023|
+-------+---------+--------+-----+----------+



In [14]:
query = """
SELECT ProductID, SUM(Quantity * Price) as TotalRevenue
FROM sales_data
GROUP BY ProductID
ORDER BY TotalRevenue DESC
limit 2
"""
result = spark.sql(query)
result.show()

+---------+------------+
|ProductID|TotalRevenue|
+---------+------------+
|      101|         500|
|      103|         250|
+---------+------------+



In [17]:
from pyspark.sql.functions import col

data = [("Alice", 34, "Female"),
        ("Bob", 45, "Male"),
        ("Charlie", 28, "Male"),
        ("Diana", 39, "Female")]
columns = ["Name", "Age", "Gender"]
df = spark.createDataFrame(data, columns)
print('Original dataframe')
df.show()
print('Selecting Name and Age without col function')
df.select('Name','Age').show()
print('Selecting Name and Age with col function')
df.select(col('Name'),col('Age')).show()
print('Selction using [] operator')
df.select(df['Name'], df['Age']).show()


Original dataframe
+-------+---+------+
|   Name|Age|Gender|
+-------+---+------+
|  Alice| 34|Female|
|    Bob| 45|  Male|
|Charlie| 28|  Male|
|  Diana| 39|Female|
+-------+---+------+

Selecting Name and Age without col function
+-------+---+
|   Name|Age|
+-------+---+
|  Alice| 34|
|    Bob| 45|
|Charlie| 28|
|  Diana| 39|
+-------+---+

Selecting Name and Age with col function
+-------+---+
|   Name|Age|
+-------+---+
|  Alice| 34|
|    Bob| 45|
|Charlie| 28|
|  Diana| 39|
+-------+---+

Selction using [] operator
+-------+---+
|   Name|Age|
+-------+---+
|  Alice| 34|
|    Bob| 45|
|Charlie| 28|
|  Diana| 39|
+-------+---+



Selection using column indices

In [19]:
col_ind = [2,0,1]
cols = [df.columns[i] for i in col_ind]
df4 = df.select(cols)
df4.show()

+------+-------+---+
|Gender|   Name|Age|
+------+-------+---+
|Female|  Alice| 34|
|  Male|    Bob| 45|
|  Male|Charlie| 28|
|Female|  Diana| 39|
+------+-------+---+



Withcolumns and drop function, adding a new columns IsMiddleAged and excluding Gender

In [22]:
df5 = df.withColumn("IsMiddleAged", col("Age") >= 30).drop("Gender")
df5.show()

+-------+---+------------+
|   Name|Age|IsMiddleAged|
+-------+---+------------+
|  Alice| 34|        true|
|    Bob| 45|        true|
|Charlie| 28|       false|
|  Diana| 39|        true|
+-------+---+------------+



Selection with sql expression

In [23]:
df.selectExpr("Name", "Age", "Age >= 18 as IsAdult").show()

+-------+---+-------+
|   Name|Age|IsAdult|
+-------+---+-------+
|  Alice| 34|   true|
|    Bob| 45|   true|
|Charlie| 28|   true|
|  Diana| 39|   true|
+-------+---+-------+

